In [ ]:
import sys
import torch 
import numpy as np


In [ ]:
# sys.path.insert(0, 'D:\\Mestrado\\Emotion Audio Synthesis (TTS)\\repo_final\\pt_etts')

In [ ]:
sys.path

# Primeiro vamos criar um exemplo de metadata pra debuggar com a base do LJSpeech


In [ ]:
import os
import pandas as pd

In [ ]:
# Pegando o caminho a partir do diretório atual que nos leva a 10 exemplos de arquivos .wav

wav_files = ['./data/LJSpeech/LJSpeech-1.1/wavs/' + f for f in os.listdir('./data/LJSpeech/LJSpeech-1.1/wavs')[:10]]
wav_files

In [ ]:
# Agora criando 10 exemplos de textos quaisquer

texts = ['hi',
        'my name is lucas',
        'and yours?',
        'my name is jessica',
        'how are you doing?',
        'fine and you?',
        'the same',
        'wanna hangout some day?',
        'yes of course',
        'ok, see ya']

speakers = ['rosana',
           'marco',
           'rosana',
           'marco',
           'rosana',
           'marco',
           'rosana',
           'marco',
           'rosana',
           'marco']

styles = ['neutral',
         'neutral',
         'neutral',
         'neutral',
         'happy',
         'happy',
         'happy',
         'happy',
         'happy',
         'happy']

len(texts), len(speakers), len(styles)

In [ ]:
df = pd.DataFrame({'wav_file':wav_files, 'text': texts, 'speakers': speakers , 'styles': styles})
df.head()

In [ ]:
df.to_csv("./experiments/debug/debug_meta.csv", sep= '|', encoding = 'latin-1', index = False)

# Agora vamos testar o dataloader

In [ ]:
import argparse
import glob
import os
import sys
import time
import traceback

import numpy as np
import torch

from random import randrange
from torch.utils.data import DataLoader
from TTS.tts.datasets.preprocess import load_meta_data
from TTS.tts.datasets.TTSDataset import MyDataset
from TTS.tts.layers.losses import TacotronLoss
from TTS.tts.utils.distribute import (DistributedSampler,
                                      apply_gradient_allreduce,
                                      init_distributed, reduce_tensor)
from TTS.tts.utils.generic_utils import setup_model, check_config_tts
from TTS.tts.utils.io import save_best_model, save_checkpoint
from TTS.tts.utils.measures import alignment_diagonal_score
from TTS.tts.utils.speakers import (get_speakers, load_speaker_mapping,
                                    save_speaker_mapping)
from TTS.tts.utils.styles import (get_styles, load_style_mapping,
                                    save_style_mapping)
from TTS.tts.utils.synthesis import synthesis
from TTS.tts.utils.text.symbols import make_symbols, phonemes, symbols
from TTS.tts.utils.visual import plot_alignment, plot_spectrogram
from TTS.utils.audio import AudioProcessor
from TTS.utils.console_logger import ConsoleLogger
from TTS.utils.generic_utils import (KeepAverage, count_parameters,
                                     create_experiment_folder, get_git_branch,
                                     remove_experiment_folder, set_init_dict)
from TTS.utils.io import copy_config_file, load_config
from TTS.utils.radam import RAdam
from TTS.utils.tensorboard_logger import TensorboardLogger
from TTS.utils.training import (NoamLR, adam_weight_decay, check_update,
                                gradual_training_scheduler, set_weight_decay,
                                setup_torch_training_env)

In [ ]:
use_cuda, num_gpus = setup_torch_training_env(True, False)

def setup_loader(ap, r, is_val=False, verbose=False, speaker_mapping=None, style_mapping = None):
    if is_val and not c.run_eval:
        loader = None
    else:
        dataset = MyDataset(
            r,
            c.text_cleaner,
            compute_linear_spec=c.model.lower() == 'tacotron',
            meta_data=meta_data_eval if is_val else meta_data_train,
            ap=ap,
            tp=c.characters if 'characters' in c.keys() else None,
            batch_group_size=0 if is_val else c.batch_group_size *
            c.batch_size,
            min_seq_len=c.min_seq_len,
            max_seq_len=c.max_seq_len,
            phoneme_cache_path=c.phoneme_cache_path,
            use_phonemes=c.use_phonemes,
            phoneme_language=c.phoneme_language,
            enable_eos_bos=c.enable_eos_bos_chars,
            verbose=verbose,
            speaker_mapping=speaker_mapping if c.use_speaker_embedding and c.use_external_speaker_embedding_file else None)
        sampler = DistributedSampler(dataset) if num_gpus > 1 else None
        loader = DataLoader(
            dataset,
            batch_size=c.eval_batch_size if is_val else c.batch_size,
            shuffle=False,
            collate_fn=dataset.collate_fn,
            drop_last=False,
            sampler=sampler,
            num_workers=c.num_val_loader_workers
            if is_val else c.num_loader_workers,
            pin_memory=False)
    return loader

def format_data(data, speaker_mapping=None, style_mapping = None):
    if speaker_mapping is None and c.use_speaker_embedding and not c.use_external_speaker_embedding_file:
        speaker_mapping = load_speaker_mapping(OUT_PATH)
    if style_mapping is None and c.use_style_embeddings:
        style_mapping = load_style_mapping(OUT_PATH)

    # setup input data
    text_input = data[0]
    text_lengths = data[1]
    speaker_names = data[2]
    linear_input = data[3] if c.model in ["Tacotron"] else None
    mel_input = data[4]
    mel_lengths = data[5]
    stop_targets = data[6]
    style_targets = data[10]
    avg_text_length = torch.mean(text_lengths.float())
    avg_spec_length = torch.mean(mel_lengths.float())

    if c.use_speaker_embedding:
        if c.use_external_speaker_embedding_file:
            speaker_embeddings = data[8]
            speaker_ids = None
        else:
            speaker_ids = [
                speaker_mapping[speaker_name] for speaker_name in speaker_names
            ]
            speaker_ids = torch.LongTensor(speaker_ids)
            speaker_embeddings = None
    else:
        speaker_embeddings = None
        speaker_ids = None

    if c.use_style_embedding:
        style_targets = [
                style_mapping[style_target] for style_target in style_targets
            ]
        if c.use_one_hot_style: # Style target will be a one hotted vector
            style_targets_ = np.zeros((len(style_targets), len(style_mapping)))
            for i in range(len(style_targets_)):
                if(style_targets[i] != 0): # If we force the 0 mapped style to be the non
                    style_targets_[i][style_targets[i]] = 1 # For each position we one hot encode it
            
            style_targets = style_targets_
            
            style_targets = torch.FloatTensor(style_targets)
            
            del style_targets_
        else: # Style target will be just the indice
            style_targets = torch.FloatTensor(style_targets)
    else:
        style_targets = None

    # set stop targets view, we predict a single stop token per iteration.
    stop_targets = stop_targets.view(text_input.shape[0],
                                     stop_targets.size(1) // c.r, -1)
    stop_targets = (stop_targets.sum(2) >
                    0.0).unsqueeze(2).float().squeeze(2)

    # dispatch data to GPU
    if use_cuda:
        text_input = text_input.cuda(non_blocking=True)
        text_lengths = text_lengths.cuda(non_blocking=True)
        mel_input = mel_input.cuda(non_blocking=True)
        mel_lengths = mel_lengths.cuda(non_blocking=True)
        linear_input = linear_input.cuda(non_blocking=True) if c.model in ["Tacotron"] else None
        stop_targets = stop_targets.cuda(non_blocking=True)
        if speaker_ids is not None:
            speaker_ids = speaker_ids.cuda(non_blocking=True)
        if speaker_embeddings is not None:
            speaker_embeddings = speaker_embeddings.cuda(non_blocking=True)
        if style_targets is not None:
            style_targets = style_targets.cuda(non_blocking=True)

    return text_input, text_lengths, mel_input, mel_lengths, linear_input, stop_targets, speaker_ids, speaker_embeddings, avg_text_length, avg_spec_length, style_targets


In [ ]:
config_path = './experiments/debug/config.json'
experiment_folder = './experiments/debug/'
c = load_config(config_path)

In [ ]:
OUT_PATH = experiment_folder

In [ ]:
ap = AudioProcessor(**c.audio)

In [ ]:
meta_data_train, meta_data_eval = load_meta_data(c.datasets)

In [ ]:
meta_data_train

In [ ]:
if 'characters' in c.keys():
    symbols, phonemes = make_symbols(**c.characters)

# DISTRUBUTED
if num_gpus > 1:
    init_distributed(args.rank, num_gpus, args.group_id,
                     c.distributed["backend"], c.distributed["url"])
num_chars = len(phonemes) if c.use_phonemes else len(symbols)

In [ ]:
restore_path = None # primeiro ele é none pra criar o json
# restore_path = experiment_folder

# parse speakers
if c.use_speaker_embedding:
    speakers = get_speakers(meta_data_train)
    if restore_path:
        if c.use_external_speaker_embedding_file: # if restore checkpoint and use External Embedding file
            prev_out_path = os.path.dirname(args.restore_path)
            speaker_mapping = load_speaker_mapping(prev_out_path)
            if not speaker_mapping:
                print("WARNING: speakers.json was not found in restore_path, trying to use CONFIG.external_speaker_embedding_file")
                speaker_mapping = load_speaker_mapping(c.external_speaker_embedding_file)
                if not speaker_mapping:
                    raise RuntimeError("You must copy the file speakers.json to restore_path, or set a valid file in CONFIG.external_speaker_embedding_file")
            speaker_embedding_dim = len(speaker_mapping[list(speaker_mapping.keys())[0]]['embedding'])
        elif not c.use_external_speaker_embedding_file: # if restore checkpoint and don't use External Embedding file
            prev_out_path = os.path.dirname(restore_path)
            speaker_mapping = load_speaker_mapping(prev_out_path)
            speaker_embedding_dim = None
            assert all([speaker in speaker_mapping
                        for speaker in speakers]), "As of now you, you cannot " \
                                                "introduce new speakers to " \
                                                "a previously trained model."
    else: # if start new train and don't use External Embedding file
        speaker_mapping = {name: i for i, name in enumerate(speakers)}
        speaker_embedding_dim = None
    save_speaker_mapping(OUT_PATH, speaker_mapping)
    num_speakers = len(speaker_mapping)
    print("Training with {} speakers: {}".format(num_speakers,
                                                 ", ".join(speakers)))
else:
    num_speakers = 0
    speaker_embedding_dim = None
    speaker_mapping = None
    
    
# parse styles
if c.use_style_embedding:
    styles = get_styles(meta_data_train)
    if restore_path:
        prev_out_path = os.path.dirname(restore_path)
        style_mapping = load_style_mapping(prev_out_path)
        style_embedding_dim = None
        assert all([style in style_mapping
                    for style in styles]), "As of now you, you cannot " \
                                            "introduce new styles to " \
                                            "a previously trained model."
    else: # if start new train and don't use External Embedding file
        style_mapping = {name: i for i, name in enumerate(styles)}
        style_embedding_dim = None
    save_style_mapping(OUT_PATH, style_mapping)
    num_styles = len(style_mapping)
    print("Training with {} styles: {}".format(num_styles,
                                                 ", ".join(styles)))
else:
    num_styles = 0
    style_embedding_dim = None
    style_mapping = None

model = setup_model(num_chars, num_speakers, c, speaker_embedding_dim)

In [ ]:
style_mapping

In [ ]:
style_mapping

In [ ]:
epoch = 0
data_loader = setup_loader(ap, model.decoder.r, is_val=False,
                           verbose=(epoch == 0), speaker_mapping=speaker_mapping, style_mapping = style_mapping)

In [ ]:

for data in data_loader:
    text_input, text_lengths, mel_input, mel_lengths, linear_input, stop_targets, speaker_ids, speaker_embeddings, avg_text_length, avg_spec_length, style_targets = format_data(data, speaker_mapping, style_mapping)
    if c.bidirectional_decoder or c.double_decoder_consistency:
        decoder_output, postnet_output, alignments, stop_tokens, decoder_backward_output, alignments_backward, logits = model.cuda()(
            text_input, text_lengths, mel_input, mel_lengths, speaker_ids=speaker_ids, speaker_embeddings=speaker_embeddings)
    else:
        decoder_output, postnet_output, alignments, stop_tokens, logits = model.cuda()(
            text_input, text_lengths, mel_input, mel_lengths, speaker_ids=speaker_ids, speaker_embeddings=speaker_embeddings)
        decoder_backward_output = None
        alignments_backward = None
    
    break

In [ ]:
from torch.nn import functional

In [ ]:
gst_logits_target = style_targets
gst_logits = logits
loss = 0
return_dict = {}
if c.gst_style_loss:
    gst_style_loss = functional.binary_cross_entropy(gst_logits.squeeze(0).squeeze(1), gst_logits_target)
    loss += gst_style_loss
    return_dict['gst_logits_loss'] = gst_style_loss

In [1]:
loss

NameError: name 'loss' is not defined

In [ ]:
! python TTS/bin/train_tts.py --config_path "./experiments/debug/config.json" --experiment_folder "./experiments/debug/" --rank 1 --continue_path ""

In [ ]:
sys.path

In [ ]:
os.path.isfile(config_path)